<a href="https://colab.research.google.com/github/Yassmina-Abdo/natural-scene-classification/blob/main/scene_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import osimport tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Concatenate, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers, activations

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Part1-Data Preprocessing

### Preprocessing the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range=0.2,
                                   horizontal_flip = True
                                   )

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Scenes_training_set/',
                                target_size=(150,150),
                                batch_size=32,
                                class_mode='categorical')


Found 12564 images belonging to 6 classes.


### Preprocessing the Testing set

In [ ]:
ts_datagen = ImageDataGenerator(rescale=1.0/255)
testing_set = ts_datagen.flow_from_directory('/content/drive/MyDrive/My_Scenes_testing_set',
                               target_size=(150,150),
                               batch_size=32,
                               class_mode='categorical',
                               )


Found 1470 images belonging to 6 classes.


## Part 2 - Building the CNN

In [ ]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=[150, 150, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))


cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))


  cnn.add(tf.keras.layers.Flatten())
  cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))
  cnn.add(tf.keras.layers.Dropout(0.4))

  cnn.add(tf.keras.layers.Dense(units=64, activation='relu'))
  cnn.add(tf.keras.layers.Dense(units=6, activation='softmax'))

In [ ]:
filepath = "model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', 
                        verbose=1, save_best_only=True, mode='max')

early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=4, restore_best_weights=True)
callbacks_list = [checkpoint, early]


cnn.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])
cnn.fit(x = training_set, validation_data = testing_set, epochs = 100, callbacks=callbacks_list) 

## Predict



In [ ]:
import os
from keras.preprocessing import image
images = []
images_name = []
folder_path = '/content/drive/MyDrive/Scenes_testing_test/'

for img in os.listdir(folder_path):
    images_name.append(img)
    img = os.path.join(folder_path, img)
    img = image.load_img(img, target_size=(150, 150))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    images.append(img)

images = np.vstack(images)

In [ ]:
cnn.load_weights('/content/drive/MyDrive/model.h5')
classes = np.argmax(cnn.predict(images), axis=-1)
images_name = np.array(images_name)
import pandas as pd
dataset = pd.DataFrame({'Image': images_name, 'Label': list(classes)}, columns=['Image', 'Label'])
print(classes)

In [ ]:
dataset.to_csv(r'predictions.csv', index = False)

In [ ]:
dataset.Label.value_counts()